<a href="https://colab.research.google.com/github/Ramakrishna-C/LSTM-CNN-implementation-for-BCI-iv/blob/main/SummerInternship.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#imports
import torch
import torch.nn as nn
import torch.optim as optim 
import torch.nn.functional as F #can use nn as well, for fucntions with no parameter
from torch.utils.data import DataLoader #gives mini batches to train on
import torchvision.datasets as datasets #standard datasets
import torchvision.transforms as transforms #several transformations that we can perform on datasets

In [ ]:
#create fully connected network
class NN(nn.Module):
  def __init__(self, input_size, num_classes):
    super(NN, self).__init__()
    self.fc1=nn.Linear(input_size, 50)
    self.fc2=nn.Linear(50, num_classes)

  def forward(self, x):
    x = F.relu(self.fc1(x))
    x = self.fc2(x)
    return x

In [3]:
#CNN
class CNN(nn.Module):
  def __init__(self, in_channels = 1, num_classes = 10):
    super(CNN, self).__init__()
    self.conv1 = nn.Conv2d(in_channels = 1, out_channels = 8, kernel_size = (3,3), stride = (1,1), padding = (1,1) )  #8 is arbitrary kernel, padding stride is arbitrary but theres a reason (same conv)
    self.pool = nn.MaxPool2d(kernel_size = (2,2), stride = (2,2))
    self.conv2 = nn.Conv2d(in_channels = 8, out_channels = 16, kernel_size = (3,3), stride = (1,1), padding = (1,1) )
    self.fc1 = nn.Linear(16*7*7, num_classes)
  
  def forward(self, x):
    x = F.relu(self.conv1(x))
    x = self.pool(x)
    x = F.relu(self.conv2(x))
    x = self.pool(x)
    x = x.reshape(x.shape[0], -1)
    x = self.fc1(x)

    return x


In [4]:
#basic check for model parameters
#CNN
model = CNN()
x = torch.randn(64, 1, 28, 28)
print(model(x).shape)
exit()

torch.Size([64, 10])


In [ ]:
#dont need for colab
#set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [1]:
#CNN
#Hyperparameters
in_channels = 1
num_classes = 10
learning_rate = 0.01
batch_size = 64
num_epochs = 5 

In [ ]:
#Hyperparameters
input_size = 784
num_classes = 10
learning_rate = 0.01
batch_size = 64
num_epochs = 1

In [ ]:
#Load data
train_dataset = datasets.MNIST(root = './data', train = True,
                        transform = transforms.ToTensor(), download = True)

train_loader = DataLoader(dataset = train_dataset, batch_size=batch_size, shuffle = True)


test_dataset = datasets.MNIST(root = './data', train = False,
                        transform = transforms.ToTensor(), download = True)

test_loader = DataLoader(dataset = test_dataset, batch_size=batch_size, shuffle = True)

In [ ]:
#initialsize the network
model = NN(input_size=input_size, num_classes=num_classes).to(device)

In [3]:
#CNN
#initialsize the network
model = CNN().to(device)#dont need to call anything, all default parameters used

NameError: ignored

In [ ]:
#Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
#train network
for epoch in range(num_epochs):
  for batch_idx, (data, targets) in enumerate(train_loader):
    data = data.to(device=device)
    targets = targets.to(device=device)

    #get correct shape, makes it into 64x7*4 data
    data = data.reshape(data.shape[0], -1)

    #forward
    scores = model(data)
    loss = criterion(scores, targets)

    #backward
    optimizer.zero_grad() #do this to initialize gradients to zero after every epoch
    loss.backward()

    #gradient descent
    optimizer.step()

In [ ]:
#CNN
#train network
for epoch in range(num_epochs):
  for batch_idx, (data, targets) in enumerate(train_loader):
    data = data.to(device=device)
    targets = targets.to(device=device)
    
    #already have a correct shape, dont need to reshape

    #forward
    scores = model(data)
    loss = criterion(scores, targets)

    #backward
    optimizer.zero_grad() #do this to initialize gradients to zero after every epoch
    loss.backward()

    #gradient descent
    optimizer.step()

In [ ]:
#check accuracy

def check_accuracy(loader, model):
  if loader.dataset.train:
    print("Checking accuracy on training data")
  else:
    print("Checking accuracy on test data")

  num_correct = 0
  num_samples = 0
  model.eval()

  with torch.no_grad(): #dont have to compute gradients to check accuracy
    for x,y in loader:
      x = x.to(device=device)
      y = y.to(device=device)
      x = x.reshape(x.shape[0], -1)

      scores = model(x)
      _, predictions = scores.max(1)
      num_correct += (predictions == y).sum()
      num_samples += predictions.size(0)

    model.train()
    return num_correct/num_samples 

In [ ]:
#CNN
#check accuracy

def check_accuracy(loader, model):
  if loader.dataset.train:
    print("Checking accuracy on training data")
  else:
    print("Checking accuracy on test data")

  num_correct = 0
  num_samples = 0
  model.eval()

  with torch.no_grad(): #dont have to compute gradients to check accuracy
    for x,y in loader:
      x = x.to(device=device)
      y = y.to(device=device)

      scores = model(x)
      _, predictions = scores.max(1)
      num_correct += (predictions == y).sum()
      num_samples += predictions.size(0)

    model.train()
    return num_correct/num_samples 

In [ ]:
print(f"Accuracy on training set: {check_accuracy(train_loader, model)*100:.2f}")
print(f"Accuracy on test set: {check_accuracy(test_loader, model)*100:.2f}")

Checking accuracy on training data
Accuracy on training set: 96.19
Checking accuracy on test data
Accuracy on test set: 95.74
